In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb
import pandas_profiling as ppf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import os
import gc

pd.set_option('display.max_columns', 50)
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('.\\data\\first_round_training_data.csv')
test = pd.read_csv('.\\data\\first_round_testing_data.csv')
submit_example = pd.read_csv('.\\data\\submit_example.csv')

print('shape of train:', train.shape)
print('shape of test:', test.shape)
print('shape of submit:', submit_example.shape)

shape of train: (6000, 21)
shape of test: (6000, 11)
shape of submit: (120, 5)


In [3]:
train.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Quality_label
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,6.856075,0.168761,1.098755,36.955992,8.454598,11.438066,177.243120,338.729256,2.021704,0.079526,Pass
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,0.000362,11.649033,0.066671,225.632949,0.481860,20597.447822,3.723330,15.376190,0.986973,4.634376,Fail
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,0.022201,0.078213,110.079689,2.208138,0.073525,236.079314,0.064196,0.576302,33.875790,1.813727,Fail
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,1.459004,0.380281,0.011491,0.654517,0.025872,176.948915,0.029777,0.246726,27.117165,0.081819,Fail
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,11.576647,1.555672,38.613386,0.260989,0.009380,194.798039,0.055053,0.014725,13.569707,18.138496,Fail


In [4]:
test.head().append(test.tail())

,Group,Parameter1,Parameter10,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9
0,0,0.235279,0.195680,0.351488,0.186814,0.086456,1.434060,1.230409,0.038483,2.931083,2.005852
1,0,0.043799,0.523976,32.590688,0.886285,29.702490,0.027434,0.016852,0.600827,0.118077,887.611257
2,0,21.466737,0.523976,11.847792,79499.554454,13544.605343,0.027434,0.016852,0.600827,0.118077,887.611257
3,0,0.703795,0.010192,0.018080,317.015599,38.087562,0.428092,0.314162,0.038483,17.850021,0.051850
4,0,10.447960,0.010192,15.983891,0.996049,16.715690,0.477823,0.464037,0.038483,17.850021,0.051850
5995,119,765.876721,51.944717,0.053836,0.258969,0.125856,0.000218,0.000414,2286.523413,0.035407,0.593081
5996,119,0.071211,0.195680,0.924208,284.265495,15.668770,3.095123,1.817391,0.600827,17.850021,6.783967
5997,119,0.001922,0.073078,7.829744,16.138304,0.000376,0.741630,1.495371,0.600827,17.850021,0.051850
5998,119,0.000054,0.027291,0.672029,0.002134,0.149019,3.454681,3.262468,0.600827,17.850021,6.783967
5999,119,1.026527,0.073078,0.116152,2.923321,610.091923,1.031282,0.833011,0.038483,2.931083,2.005852


In [5]:
submit_example.head()

,Group,Excellent ratio,Good ratio,Pass ratio,Fail ratio
0,0,0.4,0.2,0.2,0.2
1,1,0.4,0.2,0.2,0.2
2,2,0.4,0.2,0.2,0.2
3,3,0.4,0.2,0.2,0.2
4,4,0.4,0.2,0.2,0.2


In [6]:
train.Quality_label.unique()

array(['Pass', 'Fail', 'Good', 'Excellent'], dtype=object)

In [7]:
features = ['Parameter1', 'Parameter2', 'Parameter3', 'Parameter4', 'Parameter5',
            'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10',
            'Attribute1', 'Attribute2', 'Attribute3', 'Attribute4', 'Attribute5',
            'Attribute6', 'Attribute7', 'Attribute8', 'Attribute9', 'Attribute10']
test_features = ['Parameter1', 'Parameter10', 'Parameter2', 'Parameter3',
                 'Parameter4', 'Parameter5', 'Parameter6', 'Parameter7', 
                 'Parameter8', 'Parameter9']

### 特征类别观察

In [8]:
for feature in test_features:
    print(feature, len(train[feature].value_counts().tolist()))

Parameter1 6000
Parameter10 41
Parameter2 6000
Parameter3 6000
Parameter4 6000
Parameter5 132
Parameter6 80
Parameter7 14
Parameter8 23
Parameter9 16


In [9]:
def label_map(x):
    labels = {'Excellent':1, 'Good':2, 'Pass':3, 'Fail':4}
    return labels[x]

train['label'] = train.Quality_label.apply(label_map)
train.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Quality_label,label
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,6.856075,0.168761,1.098755,36.955992,8.454598,11.438066,177.243120,338.729256,2.021704,0.079526,Pass,3
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,0.000362,11.649033,0.066671,225.632949,0.481860,20597.447822,3.723330,15.376190,0.986973,4.634376,Fail,4
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,0.022201,0.078213,110.079689,2.208138,0.073525,236.079314,0.064196,0.576302,33.875790,1.813727,Fail,4
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,1.459004,0.380281,0.011491,0.654517,0.025872,176.948915,0.029777,0.246726,27.117165,0.081819,Fail,4
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,11.576647,1.555672,38.613386,0.260989,0.009380,194.798039,0.055053,0.014725,13.569707,18.138496,Fail,4


In [ ]:
pfr = ppf.ProfileReport(train)
pfr.to_file('train_EDA.html')

### 异常点观察

In [ ]:
plt.scatter(train.Quality_label, train.Parameter1)
# 剔除train.Parameter1>12的数据

In [ ]:
train = train[train.Parameter1<=200000]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter2)
# 剔除train.Parameter2>14的数据

In [ ]:
train = train[train.Parameter2<=1000000]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter3)
# 剔除train.Parameter3>12的数据

In [ ]:
train = train[train.Parameter3<=200000]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter4)

In [ ]:
train = train[train.Parameter4<=80000]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter5)
# 剔除train.Parameter5>4的数据

In [ ]:
train = train[train.Parameter5<=60]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter6)

In [ ]:
plt.scatter(train.Quality_label, train.Parameter7)

In [ ]:
train = train[train.Parameter7<=30000]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter8)
# 剔除train.Parameter8>8的数据

In [ ]:
train = train[train.Parameter8<=5000]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter9)
# 剔除train.Parameter9>12的数据

In [ ]:
train = train[train.Parameter9<=1e8]

In [ ]:
plt.scatter(train.Quality_label, train.Parameter10)

In [ ]:
train.shape

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(train[test_features].corr(), vmax=0.8, vmin=-0.8, linewidths=0.2, annot=True)

### 构建数据组合

In [10]:
selected_features = ['Parameter1', 'Parameter2', 'Parameter3',
                     'Parameter4', 'Parameter6', 'Parameter7', 
                     'Parameter8', 'Parameter9', 'Parameter10']
features = ['Parameter6', 'Parameter7', 'Parameter8', 
            'Parameter9', 'Parameter10']
full = pd.concat([train[selected_features], test[selected_features]], ignore_index=True)

#### Label Encoding

In [11]:
for feature in features:
    le_feature = feature + '_le'
    full[le_feature] = LabelEncoder().fit_transform(full[feature])

In [12]:
full.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Parameter6_le,Parameter7_le,Parameter8_le,Parameter9_le,Parameter10_le
0,0.001660,0.591013,147.608373,38.186345,0.000612,2286.523413,0.035407,0.593081,1.010385,24,12,1,5,24
1,1.601749,0.015052,0.035864,51.130326,0.002397,2286.523413,0.035407,0.593081,1.010385,31,12,1,5,24
2,0.098039,69.233685,0.080920,0.112265,0.001972,2286.523413,0.035407,0.593081,1.010385,30,12,1,5,24
3,18.181860,0.047325,0.018061,1.098102,0.002397,2286.523413,0.035407,0.593081,1.010385,31,12,1,5,24
4,0.012085,0.008749,0.005509,524.327396,0.002397,2286.523413,0.035407,0.593081,1.010385,31,12,1,5,24


In [13]:
trainX = full[:train.shape[0]]
trainX['label'] = train.label
testX = full[train.shape[0]:]

print('shape of trainX:', trainX.shape)
print('shape of testX:', testX.shape)

shape of trainX: (6000, 15)
shape of testX: (6000, 14)


#### Target Encoding

In [14]:
def parameter_ctr(train, test, col, label):
    # train: 数据集
    # col: 类别型特征
    # label：标签值
    new_col = col + '_ctr_' + str(label)
    train[new_col] = 0.0
    
    ctr = train[col+'_le'].value_counts() 
    k_map = {}
    values = train[col+'_le'].unique().tolist()
    values.extend(test[col+'_le'].unique().tolist())
    for elem in set(values):
        try:
            k_map[elem] = train[(train[col+'_le']==elem)&(train['label']==label)].shape[0] / ctr[elem]
        except:
            k_map[elem] = sum(k_map.values()) / len(k_map.values())
    train[new_col] = train[col+'_le'].apply(lambda x: k_map[x])
    test[new_col] = test[col+'_le'].apply(lambda x: k_map[x])
    return train, test

features = ['Parameter6', 'Parameter7', 'Parameter8', 
            'Parameter9', 'Parameter10']
labels = [1, 2, 3, 4]
for feature in features:
    for label in labels:
        print(feature, label)
        trainX, testX = parameter_ctr(trainX, testX, feature, label)

Parameter6 1
Parameter6 2
Parameter6 3
Parameter6 4
Parameter7 1
Parameter7 2
Parameter7 3
Parameter7 4
Parameter8 1
Parameter8 2
Parameter8 3
Parameter8 4
Parameter9 1
Parameter9 2
Parameter9 3
Parameter9 4
Parameter10 1
Parameter10 2
Parameter10 3
Parameter10 4


In [15]:
print('shape of trainX:', trainX.shape)
print('shape of testX:', testX.shape)

shape of trainX: (6000, 35)
shape of testX: (6000, 34)


In [16]:
trainX.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Parameter6_le,Parameter7_le,Parameter8_le,Parameter9_le,Parameter10_le,label,Parameter6_ctr_1,Parameter6_ctr_2,Parameter6_ctr_3,Parameter6_ctr_4,Parameter7_ctr_1,Parameter7_ctr_2,Parameter7_ctr_3,Parameter7_ctr_4,Parameter8_ctr_1,Parameter8_ctr_2,Parameter8_ctr_3,Parameter8_ctr_4,Parameter9_ctr_1,Parameter9_ctr_2,Parameter9_ctr_3,Parameter9_ctr_4,Parameter10_ctr_1,Parameter10_ctr_2,Parameter10_ctr_3,Parameter10_ctr_4
0,0.001660,0.591013,147.608373,38.186345,0.000612,2286.523413,0.035407,0.593081,1.010385,24,12,1,5,24,3,0.000000,0.222222,0.555556,0.222222,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
1,1.601749,0.015052,0.035864,51.130326,0.002397,2286.523413,0.035407,0.593081,1.010385,31,12,1,5,24,4,0.141509,0.386792,0.216981,0.254717,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
2,0.098039,69.233685,0.080920,0.112265,0.001972,2286.523413,0.035407,0.593081,1.010385,30,12,1,5,24,4,0.163636,0.345455,0.236364,0.254545,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
3,18.181860,0.047325,0.018061,1.098102,0.002397,2286.523413,0.035407,0.593081,1.010385,31,12,1,5,24,4,0.141509,0.386792,0.216981,0.254717,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
4,0.012085,0.008749,0.005509,524.327396,0.002397,2286.523413,0.035407,0.593081,1.010385,31,12,1,5,24,4,0.141509,0.386792,0.216981,0.254717,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895


In [17]:
drop_cols = ['Parameter1', 'Parameter2', 'Parameter3', 'Parameter4',
             'Parameter6_le', 'Parameter7_le', 'Parameter8_le', 
             'Parameter9_le', 'Parameter10_le']
trainX.drop(drop_cols, axis=1, inplace=True)
testX.drop(drop_cols, axis=1, inplace=True)

#### Frequency Encoding

In [ ]:
full['Parameter6_fre'] = LabelEncoder().fit_transform(full.Parameter6)

def parameter6_fre(x):
    ct = full.Parameter6_fre.value_counts() / full.shape[0]
    return ct[x]
full['Parameter6_fre'] = full.Parameter6_fre.apply(parameter6_fre)

In [ ]:
full['Parameter7_fre'] = LabelEncoder().fit_transform(full.Parameter7)

def parameter7_fre(x):
    ct = full.Parameter7_fre.value_counts() / full.shape[0]
    return ct[x]
full['Parameter7_fre'] = full.Parameter7_fre.apply(parameter7_fre)

In [ ]:
full['Parameter8_fre'] = LabelEncoder().fit_transform(full.Parameter8)

def parameter8_fre(x):
    ct = full.Parameter8_fre.value_counts() / full.shape[0]
    return ct[x]
full['Parameter8_fre'] = full.Parameter8_fre.apply(parameter8_fre)

In [ ]:
full['Parameter9_fre'] = LabelEncoder().fit_transform(full.Parameter9)

def parameter9_fre(x):
    ct = full.Parameter9_fre.value_counts() / full.shape[0]
    return ct[x]
full['Parameter9_fre'] = full.Parameter9_fre.apply(parameter9_fre)

In [ ]:
full['Parameter10_fre'] = LabelEncoder().fit_transform(full.Parameter10)

def parameter10_fre(x):
    ct = full.Parameter10_fre.value_counts() / full.shape[0]
    return ct[x]
full['Parameter10_fre'] = full.Parameter10_fre.apply(parameter10_fre)

In [ ]:
full2 = full.copy()

In [ ]:
full2.head()

#### 取log

In [18]:
features = ['Parameter6', 'Parameter7', 'Parameter8', 
            'Parameter9', 'Parameter10']
for feature in features:
    trainX[feature] = np.log(trainX[feature])
    testX[feature] = np.log(testX[feature])

In [19]:
full2.head()

NameError: name 'full2' is not defined

#### 数据分割

In [20]:
trainY = trainX.label
trainX.drop(['label'], axis=1, inplace=True)

print('shape of trainX:', trainX.shape)
print('shape of trainY:', trainY.shape)
print('shape of testX:', testX.shape)

shape of trainX: (6000, 25)
shape of trainY: (6000,)
shape of testX: (6000, 25)


In [30]:
trainX.drop(['Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10'], axis=1, inplace=True)
trainX.head()

,Parameter6_ctr_1,Parameter6_ctr_2,Parameter6_ctr_3,Parameter6_ctr_4,Parameter7_ctr_1,Parameter7_ctr_2,Parameter7_ctr_3,Parameter7_ctr_4,Parameter8_ctr_1,Parameter8_ctr_2,Parameter8_ctr_3,Parameter8_ctr_4,Parameter9_ctr_1,Parameter9_ctr_2,Parameter9_ctr_3,Parameter9_ctr_4,Parameter10_ctr_1,Parameter10_ctr_2,Parameter10_ctr_3,Parameter10_ctr_4
0,0.000000,0.222222,0.555556,0.222222,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
1,0.141509,0.386792,0.216981,0.254717,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
2,0.163636,0.345455,0.236364,0.254545,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
3,0.141509,0.386792,0.216981,0.254717,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895
4,0.141509,0.386792,0.216981,0.254717,0.12,0.328571,0.342857,0.208571,0.115016,0.222843,0.510783,0.151358,0.13731,0.226262,0.504059,0.132369,0.175439,0.251462,0.415205,0.157895


In [32]:
testX.drop(['Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10'], axis=1, inplace=True)
testX.head()

,Parameter6_ctr_1,Parameter6_ctr_2,Parameter6_ctr_3,Parameter6_ctr_4,Parameter7_ctr_1,Parameter7_ctr_2,Parameter7_ctr_3,Parameter7_ctr_4,Parameter8_ctr_1,Parameter8_ctr_2,Parameter8_ctr_3,Parameter8_ctr_4,Parameter9_ctr_1,Parameter9_ctr_2,Parameter9_ctr_3,Parameter9_ctr_4,Parameter10_ctr_1,Parameter10_ctr_2,Parameter10_ctr_3,Parameter10_ctr_4
6000,0.300000,0.333333,0.250000,0.116667,0.230303,0.395960,0.187879,0.185859,0.267123,0.360731,0.168950,0.203196,0.320261,0.339869,0.196078,0.143791,0.121760,0.309770,0.154973,0.413497
6001,0.103448,0.310345,0.275862,0.310345,0.298108,0.330072,0.236791,0.135029,0.049080,0.269939,0.325153,0.355828,0.076159,0.228477,0.278146,0.417219,0.245000,0.215000,0.415000,0.125000
6002,0.103448,0.310345,0.275862,0.310345,0.298108,0.330072,0.236791,0.135029,0.049080,0.269939,0.325153,0.355828,0.076159,0.228477,0.278146,0.417219,0.245000,0.215000,0.415000,0.125000
6003,0.304348,0.326087,0.282609,0.086957,0.230303,0.395960,0.187879,0.185859,0.295400,0.361380,0.205206,0.138015,0.216511,0.319771,0.342579,0.121138,0.079365,0.492063,0.142857,0.285714
6004,0.174419,0.325581,0.302326,0.197674,0.230303,0.395960,0.187879,0.185859,0.295400,0.361380,0.205206,0.138015,0.216511,0.319771,0.342579,0.121138,0.079365,0.492063,0.142857,0.285714


In [ ]:
trainX = full2[:train.shape[0]]
trainY = train.label
testX = full2[train.shape[0]:]

print('shape of trainX:', trainX.shape)
print('shape of trainY:', trainY.shape)
print('shape of testX:', testX.shape)

#### 划分训练集和验证集

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
X_train.shape

In [ ]:
model = CatBoostClassifier(iterations=2000, depth=8, learning_rate=0.01, verbose=100, loss_function='MultiClass', random_state=42)
model.fit(X_train, y_train)

predY = model.predict(X_test)

In [ ]:
model = CatBoostClassifier(iterations=2000, depth=8, learning_rate=0.01, verbose=1000, loss_function='MultiClass', random_state=42)

param = {'depth':[6, 8, 10], 'l2_leaf_reg':[0.01, 0.1]}
cv = GridSearchCV(model, param_grid=param, verbose=1, cv=5, scoring='accuracy')
cv.fit(trainX, trainY)

In [ ]:
cv.best_params_

In [ ]:
cv.best_score_

In [ ]:
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)
predY = model.predict(X_test)

In [ ]:
sum(np.array(y_test.tolist()).reshape(-1,1) == predY) / predY.shape[0]

### 模型构建

#### LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(trainX, trainY)
predY = lr.predict(testX)
proba = lr.predict_proba(testX)

#### AdaBoostClassifier

In [ ]:
ada = AdaBoostClassifier()
ada.fit(trainX, trainY)
predY = ada.predict(testX)

#### xgboost

In [ ]:
xgbc = xgb.XGBClassifier()
xgbc.fit(trainX, trainY)
predY = xgbc.predict(testX)

#### lightgbm

In [ ]:
lgbc = lgb.LGBMClassifier()
lgbc.fit(trainX, trainY)
predY = lgbc.predict(testX)

#### catboost

In [33]:
model = CatBoostClassifier(iterations=4000, depth=10, learning_rate=0.01, 
                           verbose=500, loss_function='MultiClass', 
                           random_state=42, l2_leaf_reg=0.01)
model.fit(trainX, trainY)
predY = model.predict(testX)

0:	learn: 1.3807944	total: 242ms	remaining: 16m 8s
500:	learn: 1.0042194	total: 1m 26s	remaining: 10m 5s
1000:	learn: 0.9697643	total: 2m 35s	remaining: 7m 45s
1500:	learn: 0.9444332	total: 3m 50s	remaining: 6m 24s
2000:	learn: 0.9234897	total: 5m 22s	remaining: 5m 22s
2500:	learn: 0.9089914	total: 7m 3s	remaining: 4m 13s
3000:	learn: 0.8955110	total: 8m 49s	remaining: 2m 56s
3500:	learn: 0.8846080	total: 10m 35s	remaining: 1m 30s
3999:	learn: 0.8764945	total: 12m 21s	remaining: 0us


_______

In [ ]:
proba = model.predict_proba(testX)
proba

In [34]:
test['pred'] = predY
test['prob_Excellent'] = 0.0
test['prob_Good'] = 0.0
test['prob_Pass'] = 0.0
test['prob_Fail'] = 0.0
test.loc[:, ['prob_Excellent', 'prob_Good', 'prob_Pass', 'prob_Fail']] = model.predict_proba(testX)
test.head(10)

,Group,Parameter1,Parameter10,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,pred,prob_Excellent,prob_Good,prob_Pass,prob_Fail
0,0,0.235279,0.195680,0.351488,0.186814,0.086456,1.434060,1.230409,0.038483,2.931083,2.005852,4.0,0.081581,0.274130,0.067521,0.576768
1,0,0.043799,0.523976,32.590688,0.886285,29.702490,0.027434,0.016852,0.600827,0.118077,887.611257,4.0,0.017449,0.238530,0.176502,0.567519
2,0,21.466737,0.523976,11.847792,79499.554454,13544.605343,0.027434,0.016852,0.600827,0.118077,887.611257,4.0,0.017449,0.238530,0.176502,0.567519
3,0,0.703795,0.010192,0.018080,317.015599,38.087562,0.428092,0.314162,0.038483,17.850021,0.051850,2.0,0.073558,0.645500,0.025147,0.255795
4,0,10.447960,0.010192,15.983891,0.996049,16.715690,0.477823,0.464037,0.038483,17.850021,0.051850,2.0,0.078181,0.625190,0.048883,0.247746
5,0,733.439920,0.010192,99.698641,3.202776,31.368042,0.477823,0.464037,0.038483,17.850021,0.051850,2.0,0.078181,0.625190,0.048883,0.247746
6,0,0.149962,0.010192,2.910066,2.187656,331.744593,0.533331,0.464037,0.038483,17.850021,0.051850,2.0,0.078181,0.625190,0.048883,0.247746
7,0,0.060635,0.010192,0.016492,0.007373,7183.436876,0.533331,0.464037,0.038483,17.850021,0.051850,2.0,0.078181,0.625190,0.048883,0.247746
8,0,971.284623,0.010192,29.954312,3.639672,0.136383,0.477823,0.381815,0.038483,17.850021,0.051850,2.0,0.053576,0.639532,0.015028,0.291863
9,0,1372.664549,0.010192,2.103451,0.531944,5.802723,0.477823,0.464037,0.038483,17.850021,0.051850,2.0,0.078181,0.625190,0.048883,0.247746


In [35]:
result = test.groupby(['Group'], as_index=False)['prob_Excellent', 'prob_Good', 'prob_Pass', 'prob_Fail'].mean()
result.columns = ['Group','Excellent ratio','Good ratio','Pass ratio','Fail ratio']
result.to_csv('submission17.csv', index=False)

_________

In [26]:
20000/14584 * 11000

15085.02468458585